In [ ]:
from google.colab import drive
drive.mount('/content/drive')

root_path = '/content/drive/MyDrive/learning/nus/cs5260/project'
data_path = root_path + '/data'
cleaned_data_path = root_path + '/data/cleaned'

Mounted at /content/drive


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from datasets import load_dataset

In [ ]:
dataset = load_dataset('talby/spamassassin')
emails = dataset['train'].to_pandas()
emails['text'].astype(str)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for talby/spamassassin contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/talby/spamassassin
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  wa

Generating train split: 0 examples [00:00, ? examples/s]

0        "\n> From: fork-admin@xent.com [mailto:fork-ad...
1        "Hi,\n\nOn Sun, 01 Sep 2002 00:05:03 MDT Reg C...
2        "On Fri Sep 13 2002 at 02:03, Robert Elz wrote...
3        "On Thu, 2002-08-29 at 01:06, Matthias Saou wr...
4        "Hi, I'm building an rpm for the resin webserv...
                               ...                        
10744    [" \t Preferred Non-Smoker\n \t\n Just what th...
10745    "ATTN:SIR/MADAN      \n\n                     ...
10746    "Dear Subscriber,\n\nIf I could show you a way...
10747    "****Mid-Summer Customer Appreciation SALE!***...
10748    "NEW PRODUCT ANNOUNCEMENT\n\nFrom: OUTSOURCE E...
Name: text, Length: 10749, dtype: object

In [ ]:
emails['label'].value_counts()

label
1    6954
0    3795
Name: count, dtype: int64

In [ ]:
emails['spam'] = 1 - emails['label']
emails['spam'].value_counts()

spam
0    6954
1    3795
Name: count, dtype: int64

In [ ]:
emails[emails['spam'] == 0]['group'].unique()

array(['easy_ham', 'hard_ham', 'easy_ham_2'], dtype=object)

In [ ]:
emails[emails['spam'] == 1]['group'].unique()

array(['spam', 'spam_2'], dtype=object)

In [ ]:
emails['original_text'] = emails['text']

In [ ]:
# Drop column label and group
emails = emails.drop(['label', 'group'], axis=1)

In [ ]:
# Count by spam status
emails['spam'].value_counts()

spam
0    6954
1    3795
Name: count, dtype: int64

In [ ]:
duplicate_emails = emails[emails.duplicated(subset='text', keep=False)]
duplicate_emails.sort_values(by='text')

print('Before removing duplicate emails:', emails.shape)

emails.drop_duplicates(inplace=True)
print('After removing duplicate emails:', emails.shape)

duplicate_emails.sort_values(by='text')

Before removing duplicate emails: (10749, 3)
After removing duplicate emails: (6098, 3)


,text,spam,original_text
7399,""" * WORD A DAY *\n-------...",0,""" * WORD A DAY *\n-------..."
2594,""" * WORD A DAY *\n-------...",0,""" * WORD A DAY *\n-------..."
3400,""" |:::::::::::::::::::::::::::::::::::...",0,""" |:::::::::::::::::::::::::::::::::::..."
886,""" |:::::::::::::::::::::::::::::::::::...",0,""" |:::::::::::::::::::::::::::::::::::..."
7398,""" Forgotten History - Tuesday, August 2...",0,""" Forgotten History - Tuesday, August 2..."
...,...,...,...
9750,"[[""\n\n\n\nAT LAST VIAGRA, XENICAL (And many o...",1,"[[""\n\n\n\nAT LAST VIAGRA, XENICAL (And many o..."
3064,"[[""\r\n<html>\r\n\r\n<head>\r\n<META HTTP-EQUI...",1,"[[""\r\n<html>\r\n\r\n<head>\r\n<META HTTP-EQUI..."
7707,"[[""\r\n<html>\r\n\r\n<head>\r\n<META HTTP-EQUI...",1,"[[""\r\n<html>\r\n\r\n<head>\r\n<META HTTP-EQUI..."
823,"[[""i'm a very happy user of exmh, but i'm para...",0,"[[""i'm a very happy user of exmh, but i'm para..."


In [ ]:
import ast

def parse_and_concatenate(value):
    try:
        # Attempt to parse the string as a list
        parsed_value = ast.literal_eval(value)
        if isinstance(parsed_value, list):
            # Join the list elements into a single string if parsing is successful
            return ' '.join(str(item) for item in parsed_value)
    except (ValueError, SyntaxError):
        # Return the original value if parsing fails or if it's not a list
        return value

    # Return the original value if it's not a list
    return value

# Apply the function to the DataFrame column
emails['text'] = emails['text'].apply(parse_and_concatenate)

In [ ]:
from bs4 import BeautifulSoup

def extract_text(html):
    soup = BeautifulSoup(html, "lxml")
    return soup.get_text()

emails['text'] = emails['text'].apply(extract_text)

<ipython-input-13-5df6bfd0c925>:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html, "lxml")


In [ ]:
emails['text'] = emails['text'].apply(lambda x: x.replace("\\n", " ").replace("\\r", " ").replace("\\t", " "))
emails['text'] = emails['text'].str.replace(r'\s+', ' ', regex=True).str.replace(
    '[^a-zA-Z0-9 \!\?\.,;\:\-\—\(\)\[\]\{\}\'\"%\$&\*\/=@#\\+]',
    '',
    regex=True
)

emails

,text,spam,original_text
0,""" From: fork-admin@xent.com [mailto:fork-admi...",0,"""\n> From: fork-admin@xent.com [mailto:fork-ad..."
1,"""Hi, On Sun, 01 Sep 2002 00:05:03 MDT Reg Clem...",0,"""Hi,\n\nOn Sun, 01 Sep 2002 00:05:03 MDT Reg C..."
2,"""On Fri Sep 13 2002 at 02:03, Robert Elz wrote...",0,"""On Fri Sep 13 2002 at 02:03, Robert Elz wrote..."
3,"""On Thu, 2002-08-29 at 01:06, Matthias Saou wr...",0,"""On Thu, 2002-08-29 at 01:06, Matthias Saou wr..."
4,"""Hi, I'm building an rpm for the resin webserv...",0,"""Hi, I'm building an rpm for the resin webserv..."
...,...,...,...
9348,Preferred Non-Smoker Just what the doctor orde...,1,"["" \t Preferred Non-Smoker\n \t\n Just what th..."
9349,"""ATTN:SIR/MADAN STRICTLY CONFIDENTIAL. I am pl...",1,"""ATTN:SIR/MADAN \n\n ..."
9350,"""Dear Subscriber, If I could show you a way to...",1,"""Dear Subscriber,\n\nIf I could show you a way..."
9351,"""****Mid-Summer Customer Appreciation SALE!***...",1,"""****Mid-Summer Customer Appreciation SALE!***..."


In [ ]:
emails['original_text'] = emails['text']

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize(text):
    return ' '.join(tokenizer.tokenize(text))

emails['text'] = emails['text'].apply(tokenize)

# Print the updated DataFrame
emails.head()

,text,spam,original_text
0,""" from : fork - ad ##min @ x ##ent . com [ mai...",0,"""\n> From: fork-admin@xent.com [mailto:fork-ad..."
1,""" hi , on sun , 01 sep 2002 00 : 05 : 03 md ##...",0,"""Hi,\n\nOn Sun, 01 Sep 2002 00:05:03 MDT Reg C..."
2,""" on fr ##i sep 13 2002 at 02 : 03 , robert el...",0,"""On Fri Sep 13 2002 at 02:03, Robert Elz wrote..."
3,""" on th ##u , 2002 - 08 - 29 at 01 : 06 , matt...",0,"""On Thu, 2002-08-29 at 01:06, Matthias Saou wr..."
4,""" hi , i ' m building an rpm for the resin web...",0,"""Hi, I'm building an rpm for the resin webserv..."


In [ ]:
print(emails.shape)

# Strip whitespace from the text column
emails['text'] = emails['text'].str.strip()

# Remove emails with empty text
emails = emails[emails['text'].str.len() >= 50]

print(emails.shape)

(6098, 3)
(6071, 3)


In [ ]:
emails.drop_duplicates(subset=['text', 'spam'], inplace=True)
emails.shape

(5931, 3)

In [ ]:
emails.loc[9348]['text']

'Preferred Non-Smoker Just what the doctor ordered! Case Study #1 Male - 63 $5,000,000 face Good Health 5-10 Cigarettes a day Issued: Preferred Non-Smoker Case Study #2 Female - 57 $785,000 face Good Health Social Cigarette Smoker Issued: Preferred Non-Smoker Case Study #3 Male - 52 $5,200,000 face Good Health 1-2 Cigars a month Issued:Preferred Best Non-Smoker Case Study #4 Male - 48 $1,000,000 face Private Pilot Smokes Cigars Daily Issued: Preferred Non-Smoker without Aviation Flat Extra Click here to provide the details on your tobacco case! Call the Doctor with your case details! We\'ve cured 1,000s of agents\' "tough cases!" 800-624-4502 - or - Please fill out the form below for more information Name: E-mail: Phone: City: State: Tennessee Brokerage Agency We don\'t want anyone to receive our mailings who does not wish to receive them. This is a professional communication sent to insurance professionals. To be removed from this mailing list, DO NOT REPLY to this message. Instead, g

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import words
from nltk.tokenize import word_tokenize
import re

# Ensure you have the necessary NLTK datasets downloaded
nltk.download('words')
nltk.download('punkt')
nltk.download('wordnet')

english_words = set(words.words())
lemmatizer = nltk.WordNetLemmatizer()

def count_non_english_words(text):
    # Tokenize the text into words
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text)
    tokens = word_tokenize(text)
    # Lematize tokens using lematizer
    tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens]
    # Count total words and non-English words
    total_words = len(tokens)
    non_english_words = sum(1 for word in tokens if word not in english_words)

    # Return the count of non-English words and total words
    return non_english_words, total_words

# Apply the function to the 'text' column and calculate the percentage of non-English words
emails['non_english_info'] = emails['original_text'].apply(count_non_english_words)
emails['percentage_non_english'] = emails['non_english_info'].apply(lambda x: x[0] / x[1] if x[1] > 0 else 0)

# Filter emails containing more than 30% non-English words
filtered_emails = emails[emails['percentage_non_english'] > 0.60].sort_values(by='percentage_non_english', ascending=False)

filtered_emails.shape

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


(174, 5)

In [ ]:
# Remove filtered_emails from email dataframe
print(emails.shape)
emails = emails[~emails.index.isin(filtered_emails.index)]
print(emails.shape)

(5931, 5)
(5757, 5)


In [ ]:
# Count unique words in text column
unique_words = set()
for text in emails['text']:
    unique_words.update(text.split())

print(len(unique_words))

136902


In [ ]:
emails['spam'].value_counts()

spam
0    4203
1    1554
Name: count, dtype: int64

In [ ]:
emails.sample(10)

,text,spam,original_text,non_english_info,percentage_non_english
221,""" ( this list is sponsored by iron ##cl ##ad n...",0,"""(This list is sponsored by Ironclad Networks ...","(30, 76)",0.394737
872,""" info ( 152 ##9 ##9 : 0 ) gs ##t ##x ##ml ##r...",0,"""> > > > INFO (15299: 0)gst_xml_registry_rebui...","(60, 207)",0.289855
2610,""" i am trying to secure three of four virtual ...",0,"""I am trying to secure three of four virtual h...","(35, 236)",0.148305
206,""" neat stuff . seems to combine elements of sc...",0,"""Neat stuff. Seems to combine elements of Scie...","(170, 1247)",0.136327
6357,""" for what you ' re trying to do , forward ##i...",0,""" For what you're trying to do, forwarding do...","(65, 204)",0.318627
4056,""" funny . i read it as green = red , as in acc...",0,"""funny. i read it as green = red, as in accoun...","(44, 126)",0.349206
937,""" from : paul line ##han [ mail ##to : pl ##in...",0,"""> From: Paul Linehan [mailto:plinehan@yahoo.c...","(43, 166)",0.259036
248,""" if you haven ' t already , you should enable...",0,"""If you haven't already, you should enable the...","(52, 229)",0.227074
2049,""" on mon , 26 aug 2002 , gee ##ge wrote : late...",0,"""On Mon, 26 Aug 2002, Geege wrote:\n\n> Latest...","(16, 38)",0.421053
6154,""" quoting nick mu ##rta ##gh ( nick ##m @ go #...",0,"""Quoting Nick Murtagh (nickm@go2.ie):\n\n> The...","(24, 94)",0.255319


In [ ]:
emails[['text', 'original_text', 'spam']].to_csv(cleaned_data_path + '/assassin.csv', index=False)